# DSP - Aufgabenblatt 11

## Bewertete Übung

#### FHNW - HS2019

#### Vorgaben für die Zusammenarbeit

Sie dürfen für die Lösung der Übung zusammenarbeiten, so lange sich ihre **Zusammenarbeit auf konzeptionelle und algorithmische Fragestellungen beschränkt**. Bei kopiertem Code oder Text (von Mitstudierenden oder dem Internet) werden alle Lösungen der beteiligten Parteien mit 0 Punkten bewertet. Dazu werden alle Lösungen manuell und automatisiert auf Kopien untersucht.  

#### Abgabe

Arbeiten Sie auf einem eigenen Fork des Repositories `https://gitlab.fhnw.ch/ml/courses/dsp/dsp_hs2019.git` zu welchem Sie den Dozenten (@michael.graber) als *Maintainer* hinzufügen. Committen Sie regelmässig. Als Abgabe gilt ihr **letzter Commit vor Montag, 23. Dezember 2019, 24 Uhr**.

#### Hinweise
- Der *Code* muss von Anfang bis Ende durchlaufen. Es wird nur Code bewertet, welcher beim automatischen Durchlaufen ausgeführt wird.    
- Beschriften Sie sämtliche Achsen (Bezeichnung und Einheit wenn möglich).  
- Ergänzen Sie den Plot mit zusätzlicher Beschriftung falls dies zum Verständnis des Plots notwendig ist.  
- Wenn Sie einen Plot in einer Antwort diskutieren, beginnen Sie mit einer Beschreibung dessen was der Plot zeigt.  


- Sämtliche Variablen die in einer von Ihnen definierten Funktionen verwendet werden, müssen als Argumente übergeben  oder innerhalb der Funktion definiert werden.
- Selbst definierte Funktionen müssen einen (kurzen) docstring haben.


- Versuchen Sie eine kompakte und performante Implementierung zu finden. Vollständige Punktzahl erreichen Sie bloss mit performantem *Code*.  
- Verwenden Sie dazu, wo immer möglich und sinnvoll, Funktionen aus `pandas` und `numpy`.
- Vermeiden Sie *Loops* falls möglich.

Am 14. April 1912 ist die Titanic vor der Küste Neufundlands auf ihrer Jungfernfahrt mit einem Eisberg kollidiert und sank kanpp 3 Stunden später in den frühen Morgenstunden des 15. Aprils. Von den ca 2224 Passagieren und Crewmitgliedern starben rund 1500.  

Auf der Website https://www.encyclopedia-titanica.org/ können Passagier- und Crewlisten runtergeladen werden. Ich habe Ihnen zwei Dateien bereitgestellt: 'data/titanic_survivors.csv' und 'data/titanic_victims.csv'. Sie beinhalten jeweils die Überlebenden und Opfer des Unglücks.  

Wir werden in dieser Übung die Personen hinter diesem Unglück miteinander untersuchen.

### Aufgabe 1 (10 Punkte)  

Schreiben Sie eine Funktion, mit welcher Sie die beiden Dateien laden und präprozessieren können.  

Die Funktion soll ein `DataFrame` zurückgeben mit folgenden Spalten: `'name', 'sex', 'age', 'is_adult', 'job', 'class_dept', 'is_passenger', 'joined', 'boat_body', 'ticketprice', 'survived',`.  


- Der Ticketpreis steht bei Passagieren jeweils auf der folgenden Zeile im Feld `Ticket`. Extrahieren Sie Pfund, Shilling und Pence mittels *regular expression-matching* auf dem `DataFrame`.
- Berechnen Sie einen Dezimalbetrag in Pfund gemäss dem folgenden Satz: *'There were 12 pence in a shilling and 20 shillings in a pound.'* 
- Leiten Sie das Geschlecht `sex` aus dem Namen, insbesondere aus der Anrede, ab. Finden Sie einen geeignete `pandas`-Methode für das *Mapping*.
- Berechnenen Sie das Alter für alle Personen in Jahren (`m` steht für *months*).
- `is_passenger` soll `True` sein, falls `class_dept` in `['1st Class Passenger', '2nd Class Passenger', '3rd Class Passenger',]` ist.  
- Sortieren Sie das `DataFrame` nach dem Familiennamen.

Jeder *for-Loop* gibt 2 Punkte Abzug.  

Laden Sie die Daten und geben Sie die ersten 10 Zeilen aus.

In [4]:
# imports
import pandas as pd
import numpy as np
import re

In [ ]:
# Funktion CSV-File loader
def loadFile(filepath):
    df = pd.read_csv(filepath, skiprows=5, sep=',', usecols=range(7))
    return df

In [ ]:
# load File of titanic_victims
titanic_victims_df = loadFile('../../data/titanic_victims.csv')
titanic_victims_df.head(5)
print(len(titanic_victims_df))

In [ ]:
# load File of titanic_survivors
titanic_survivors_df = loadFile('../../data/titanic_survivors.csv')
titanic_survivors_df.tail(5)
print(len(titanic_survivors_df))

In [5]:
titanic_victims_df = '../../data/titanic_victims.csv'
titanic_survivors_df = '../../data/titanic_survivors.csv'

In [22]:
def cleanFile(df_1, df_2):
    #Shift ticketprice and drop unused row
    df_1 = pd.read_csv(df_1, skiprows=5, sep=',', usecols=range(7))
    df_2 = pd.read_csv(df_2, skiprows=5, sep=',', usecols=range(7))
    
    df_1.insert(3, 'Ticketprice', np.NaN)
    ticket_prices = df_1.loc[df_1['Ticket'].str.contains('£|s|d') == True, ['Ticketprice']] = df_1['Ticket']
    df_1['Ticketprice'] = df_1['Ticketprice'].shift(-1)
    df_1 = df_1.dropna(subset=['Name'])
    
    df_2.insert(3, 'Ticketprice', np.NaN)
    ticket_prices = df_2.loc[df_2['Ticket'].str.contains('£|s|d') == True, ['Ticketprice']] = df_2['Ticket']
    df_2['Ticketprice'] = df_2['Ticketprice'].shift(-1)
    df_2 = df_2.dropna(subset=['Name'])
    
    # drop not neccassary row with NaN
    df_1 = df_1.dropna(subset=['Name'])
    df_2 = df_2.dropna(subset=['Name']) 
    
    #create new column for survived
    df_1.insert(8, 'Survived', False)
    df_2.insert(8, 'Survived', True)
    
    #Merge two files
    frames = [df_1, df_2]
    result_df = pd.concat(frames, ignore_index=True, sort=False)
    
    #drop Ticket
    result_df = result_df.drop(columns=['Ticket'])
    
    # Calculate a decimal amount in pounds
    
    
    #Rename columns
    result_df.rename(columns={'Name': 'name', 'Age': 'age', 'Class/Dept': 'class_dept', 'Ticketprice': 'ticketprice', 'Joined': 'joined',
                       'Job': 'job', 'Boat [Body]': 'boat_body', 'Survived': 'survived'}, inplace=True)
    
    #Replace Nan-Values
    values = {'job':'unknown', 'boat_body':'unknown'}
    result_df = result_df.fillna(value=values)
    
    #clean Age column with years (m=months)
    result_Age = result_df.loc[result_df['age'].str.contains('m') == True, ['age']] = round((1/12) * result_df['age'].str.split('m').str[0].astype('float64'), 1)
    
    #insert new columns
    result_df.insert(1, 'sex', np.NaN)
    result_df.insert(3, 'is_adult', np.NaN)
    result_df.insert(6, 'is_passenger', np.NaN)
    
    # Give is_Adult values
    result_df['age']= result_df['age'].astype(float)
    is_adult = result_df.loc[result_df['age'] > 17, ['is_adult']] = True
    is_adult = result_df.loc[result_df['age'] < 18, ['is_adult']] = False
    #result_df['is_adult'] = result_df.loc[result_df['age'] < 18, ['is_adult']] == False
    
    
    # is_passenger soll True sein, falls class_dept in ['1st Class Passenger', '2nd Class Passenger', '3rd Class Passenger',] ist.
    result_df['is_passenger'] = result_df['class_dept'].isin(['1st Class Passenger', '2nd Class Passenger', '3rd Class Passenger'])
    
    # split name for salutation
    result_df.insert(0, 'salutation', np.NaN)
    result_df['salutation' ]= result_df['name'].str.split(',', n=1, expand = False).str[1]
    result_df['salutation'] = result_df['salutation'].str.split(' ', n=1, expand = False).str[0]
    result_df['salutation'] = result_df['salutation'].str.extract(pat = '([A-Z][a-z]*)')
    
    
    # Mapping for sex
    result_df['sex'] = result_df['salutation'].map({'Mr': 'Male', 'Master': 'Male', 'Major': 'Male', 'Dr': 'Male', 'Sig': 'Male', 'Colonel': 'Male',
                                                      'Revd': 'Male', 'Fr': 'Male', 'Rev': 'Male', 'Captain': 'Male', 'Sr': 'Male', 'Don': 'Male', 
                                                      'Col': 'Male', 'Sir': 'Male', 'Mme': 'Female', 'Lucy':'Female', 'Sra': 'Female', 
                                                      'Mlle': 'Female', 'Mrs': 'Female', 'Miss': 'Female', 'Do': 'Female', 'Ms': 'Female'})
    
    # +4 Male and -4 Female
    result_df.at[1143,'sex'] = result_df.at[1143,'sex']= 'Female'
    print(result_df.at[1143,'sex'])
    print(len(result_df))
    
    # delete not needed column salutation
    result_df = result_df.drop(columns=['salutation'])
    
    # sort dataframe --> familyname
    cleaned_result_df = result_df.sort_values(by=['name'], ascending=True)
    cleaned_result_df.to_csv('titanic.csv')
    #print(cleaned_titanic_df['sex'].value_counts())
    print(len(cleaned_result_df))
    
    
    return cleaned_result_df

In [24]:
#import warnings
#from pandas.core.common import SettingWithCopyWarning
#warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
titanic_df = cleanFile(titanic_victims_df, titanic_survivors_df)
titanic_df.head(10)
#print(titanic_df['is_adult'].value_counts())
#print(titanic_df['is_adult'].value_counts())

Female
2208
2208


,name,sex,age,is_adult,class_dept,ticketprice,is_passenger,joined,job,boat_body,survived
0,"ABBING, Mr Anthony",Male,41.0,True,3rd Class Passenger,£7 11s,True,Southampton,Blacksmith,unknown,False
1496,"ABBOTT, Mrs Rhoda Mary 'Rosa'",Female,39.0,True,3rd Class Passenger,£20 5s,True,Southampton,unknown,A,True
3,"ABBOTT, Mr Ernest Owen",Male,21.0,True,Victualling Crew,NaN,False,Southampton,Lounge Pantry Steward,unknown,False
2,"ABBOTT, Mr Eugene Joseph",Male,13.0,False,3rd Class Passenger,£20 5s,True,Southampton,Scholar,unknown,False
1,"ABBOTT, Mr Rossmore Edward",Male,16.0,False,3rd Class Passenger,£20 5s,True,Southampton,Jeweller,[190],False
1497,"ABELSETH, Miss Karen Marie",Female,16.0,False,3rd Class Passenger,£7 13s,True,Southampton,unknown,16,True
1498,"ABELSETH, Mr Olaus Jørgensen",Male,25.0,True,3rd Class Passenger,£7 13s,True,Southampton,Farmer,A,True
1499,"ABELSON, Mrs Anna",Female,24.0,True,2nd Class Passenger,£24,True,Cherbourg,unknown,10,True
4,"ABELSON, Mr Samuel",Male,30.0,True,2nd Class Passenger,£24,True,Cherbourg,unknown,unknown,False
1500,"ABRAHAMSSON, Mr August Abraham Johannes",Male,19.0,True,3rd Class Passenger,£7 18s 6d,True,Southampton,unknown,15,True


In [ ]:
#file = '../../data/titanic_survivors.csv'
#survivors_df = pd.read_csv(file, skiprows=5, sep=',', usecols=range(7))
#print(len(survivors_df))
#
##drop unused column and switch ticketprice
#survivors_df.insert(3, 'Ticketprice', np.NaN)
#ticket_prices = survivors_df.loc[survivors_df['Ticket'].str.contains('£|s|d') == True, ['Ticketprice']] = survivors_df['Ticket']
#survivors_df['Ticketprice'] = survivors_df['Ticketprice'].shift(-1)
#
#print(len(survivors_df))
#
#survivors_df = survivors_df.dropna(subset=['Name'])
#
#print(len(survivors_df))
#
## create new column for survived?
#survivors_df.insert(8, 'Survived', True)
#
#print(len(survivors_df))
#
#survivors_df.tail(5)

In [ ]:
#file = '../../data/titanic_victims.csv'
#victims_df = pd.read_csv(file, skiprows=5, sep=',', usecols=range(7))
#
##drop unused column and switch ticketprice
#victims_df.insert(3, 'Ticketprice', np.NaN)
#ticket_prices = victims_df.loc[victims_df['Ticket'].str.contains('£|s|d') == True, ['Ticketprice']] = victims_df['Ticket']
#victims_df['Ticketprice'] = victims_df['Ticketprice'].shift(-1)
#
## drop not neccassary row with NaN
#victims_df = victims_df.dropna(subset=['Name'])
##survivors_df = survivors_df.dropna(subset=['Name']) 
## create new column for survived
#victims_df.insert(8, 'Survived', False)
##survivors_df.insert(8, 'Survived', True) --> already used above
#
#print(len(victims_df))
#print(len(survivors_df))
#
##Merge two files
#frames = [victims_df, survivors_df]
#titanic_df = pd.concat(frames, ignore_index=True, sort=False)
##print(titanic_df['Age'].unique())
#print(titanic_df['Survived'].value_counts())
#
##drop Ticket
#titanic_df = titanic_df.drop(columns=['Ticket'])
#
## Calculate a decimal amount in pounds
#
#
##Rename columns
#titanic_df.rename(columns={'Name': 'name', 'Age': 'age', 'Class/Dept': 'class_dept', 'Ticketprice': 'ticketprice', 'Joined': 'joined',
#                   'Job': 'job', 'Boat [Body]': 'boat_body', 'Survived': 'survived'}, inplace=True)
#
##Replace Nan-Values
#values = {'job':'unknown', 'boat_body':'unknown'}
#titanic_df = titanic_df.fillna(value=values)
#
##clean Age column with years (m=months)
#result_Age = titanic_df.loc[titanic_df['age'].str.contains('m') == True, ['age']] = round((1/12) * titanic_df['age'].str.split('m').str[0].astype('float64'), 1)
#
##insert new columns
#titanic_df.insert(1, 'sex', np.NaN)
#titanic_df.insert(3, 'is_adult', np.NaN)
#titanic_df.insert(6, 'is_passenger', np.NaN)
#
#print(titanic_df['sex'].value_counts())
#
## Give is_Adult values
#titanic_df['age']= titanic_df['age'].astype(float)
#titanic_df['is_adult'] = titanic_df.loc[titanic_df['age'] > 17, ['is_adult']] == True
#titanic_df['is_adult'] = titanic_df.loc[titanic_df['is_adult'] == False, ['is_adult']] == False
#
## is_passenger soll True sein, falls class_dept in ['1st Class Passenger', '2nd Class Passenger', '3rd Class Passenger',] ist.
#titanic_df['is_passenger'] = titanic_df['class_dept'].isin(['1st Class Passenger', '2nd Class Passenger', '3rd Class Passenger'])
#
## define 'sex' by name
#titanic_df.insert(0, 'salutation', np.NaN)
#titanic_df['salutation' ]= titanic_df['name'].str.split(',', n=1, expand = False).str[1]
#titanic_df['salutation'] = titanic_df['salutation'].str.split(' ', n=1, expand = False).str[0]
#titanic_df['salutation'] = titanic_df['salutation'].str.extract(pat = '([A-Z][a-z]*)')
#
#
## Mapping for sex
#titanic_df['sex'] = titanic_df['salutation'].map({'Mr': 'Male', 'Master': 'Male', 'Major': 'Male', 'Dr': 'Male', 'Sig': 'Male', 'Colonel': 'Male',
#                                                  'Revd': 'Male', 'Fr': 'Male', 'Rev': 'Male', 'Captain': 'Male', 'Sr': 'Male', 'Don': 'Male', 
#                                                  'Col': 'Male', 'Sir': 'Male', 'Mme': 'Female', 'Lucy':'Female', 'Sra': 'Female', 
#                                                  'Mlle': 'Female', 'Mrs': 'Female', 'Miss': 'Female', 'Do': 'Female', 'Ms': 'Female'})
#
#print(titanic_df['sex'].value_counts())
#
## +4 Male and -4 Female
##titanic_df.at[1143,'sex'] = titanic_df.at[1143,'sex']= 'Female'
##print(titanic_df.at[1143,'sex'])
#
## delete not needed column salutation
##titanic_df = titanic_df.drop(columns=['salutation'])
#
## sort dataframe --> familyname
#cleaned_titanic_df = titanic_df.sort_values(by=['name'], ascending=True)
#cleaned_titanic_df.to_csv('titanic.csv')
#print(len(cleaned_titanic_df))
#
#cleaned_titanic_df.head(5)

### Aufgabe 2 (8 Punkte)

Wieviele Personen sind es insgesamt?  

Wieviele Personen ohne Ticketpreis?  

Wieviele Personen ohne Altersangabe?  

Wieviele Personen männlichen und weiblichen Geschlechts fanden sich gemäss unserem Datensatz auf der Titanic?  

Wie viele Kinder, welchen Geschlechts?  

Was war der durchschnittliche Preis für eine Fahrkarte?  

Was war das Durchschnittsalter der Besatzungsmitglieder? Was war das Durchschnittsalter der Passagiere?

In [52]:
print('Es sind insgesamt', len(titanic_df['is_passenger']), 'Personen')
print('Es gibt insgesamt', titanic_df['ticketprice'].isna().sum(), 'Personen ohne Tickets')
print('Es gibt insgesamt', titanic_df['is_adult'].isna().sum(), 'Personen die keine Altersangabe haben')
print('Es gibt insgesamt', titanic_df['is_adult'].isna().sum(), 'Personen die keine Altersangabe haben')
print(titanic_df['sex'].value_counts())
print(titanic_df['is_adult'].value_counts())
titanic_df.head(10)

Es sind insgesamt 2208 Personen
Es gibt insgesamt 917 Personen ohne Tickets
Es gibt insgesamt 3 Personen die keine Altersangabe haben
Es gibt insgesamt 3 Personen die keine Altersangabe haben
Male      1720
Female     488
Name: sex, dtype: int64
True     2002
False     203
Name: is_adult, dtype: int64


,name,sex,age,is_adult,class_dept,ticketprice,is_passenger,joined,job,boat_body,survived
0,"ABBING, Mr Anthony",Male,41.0,True,3rd Class Passenger,£7 11s,True,Southampton,Blacksmith,unknown,False
1496,"ABBOTT, Mrs Rhoda Mary 'Rosa'",Female,39.0,True,3rd Class Passenger,£20 5s,True,Southampton,unknown,A,True
3,"ABBOTT, Mr Ernest Owen",Male,21.0,True,Victualling Crew,NaN,False,Southampton,Lounge Pantry Steward,unknown,False
2,"ABBOTT, Mr Eugene Joseph",Male,13.0,False,3rd Class Passenger,£20 5s,True,Southampton,Scholar,unknown,False
1,"ABBOTT, Mr Rossmore Edward",Male,16.0,False,3rd Class Passenger,£20 5s,True,Southampton,Jeweller,[190],False
1497,"ABELSETH, Miss Karen Marie",Female,16.0,False,3rd Class Passenger,£7 13s,True,Southampton,unknown,16,True
1498,"ABELSETH, Mr Olaus Jørgensen",Male,25.0,True,3rd Class Passenger,£7 13s,True,Southampton,Farmer,A,True
1499,"ABELSON, Mrs Anna",Female,24.0,True,2nd Class Passenger,£24,True,Cherbourg,unknown,10,True
4,"ABELSON, Mr Samuel",Male,30.0,True,2nd Class Passenger,£24,True,Cherbourg,unknown,unknown,False
1500,"ABRAHAMSSON, Mr August Abraham Johannes",Male,19.0,True,3rd Class Passenger,£7 18s 6d,True,Southampton,unknown,15,True


### Aufgabe 3 (3 Punkte)

Erstellen Sie eine Kreuztabelle von Einstiegsort und `class_dept`.  

Visualisieren Sie die Tabelle.  
Wir wissen, dass die Titanic von *Belfast*, wo sie gebaut wurde, nach *Southhampton*, nach *Cherbourg* und dann nach *Queenstown* fuhr, bevor Sie zur Atlantiküberfahrt auslief.  

Diskutieren Sie Ihre Erkenntnisse.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

### Aufgabe 4 (5 Punkte)

Visualisieren Sie die Altersverteilung für die beiden Geschlechter und gleichzeitig für Variable `class_dept` mit Boxplots.  

Diskutieren Sie die Visualisierung.
- Erklären Sie was ein Boxplot zeigt.
- Was für Information fehlt?
- Was können wir zu den vorliegenden Daten aus dieser Darstellung lesen?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

### Aufgabe 5 (4 Punkte)

Zeichnen Sie die Verteilung des Alters sämtlicher Passagiere als Histogramm.  

Beschreiben Sie die Verteilung bezüglich Lage, Streuung und Form.  

Sie dürfen verschiedene Ansätze verwenden und deren Resultate und Eigenschaften diskutieren.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

### Aufgabe 6 (3 Punkte)

Führen Sie die Schritte aus Aufgabe 5 für den Ticketpreis aus.  

Visualisieren Sie zusätzlich die kumulative Verteilung.  

Diskutieren Sie Ihre Erkenntnisse.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

### Aufgabe 7 (3 Punkte)

Was sind die häufigsten 10 Berufe der Passagiere, welche oberhalb des $Q_{0.9}$-Quantils des Ticketpreises liegen?  

Was sind die häufigsten 10 Berufe der Passagiere, welche unterhalb des $Q_{0.1}$-Quantils des Ticketpreises liegen?  

Wie oft kommen die Berufe jeweils vor?  
Wie interpretieren Sie das Resultat?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

### Aufgabe 8 (6 Punkte)

Finden Sie ein paar wenige Visualisierungen, welche es ermöglichen, kompakt einen Eindruck zu bekommen, wer das Unglück der Titanic am wahrscheinlichsten überlebt hat.  

Was ist ihre Erkenntnis?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE

### Aufgabe 9 (6 Punkte)

Untersuchen Sie nun, ob das Alter einen Einfluss auf die Überlebenswahrscheinlichkeit hatte.  

Teilen Sie die Passagiere dazu in Gruppen von Geschlecht und Altersjahrzenten ein.  
Berechnen Sie für die jeweiligen Gruppen den Anteil der Überlebenden, sowie den [*Standard Error of the Mean* (SEM)](https://en.wikipedia.org/wiki/Standard_error).  

Visualisieren Sie die Resultate übersichtlich und diskutieren Sie diese.  
Was können wir durch die Visualisierung des SEM erkennen?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

YOUR ANSWER HERE